In [3]:
# Code to evaluate EPMC annotation to the Manual Annotation
# Code to compare results from ML methods and EPMC annotation

# (c) EMBL-EBI, September 2019
#
# Started: 23 Septmember  2019
# Updated: 24 Septmember  2019

_author_ = 'Santosh Tirunagari'

import os
import pandas as pd
import glob
import json
import csv
import sys

import multiprocessing

import numpy as np
import re

from nltk.tokenize import wordpunct_tokenize

import requests
# from pprint import pprint
import pandas as pd

from collections import defaultdict, Counter
import time
from requests.compat import urljoin

from tqdm import tqdm

In [2]:
# Generate train, test and dev pmc ids from the random seed set to 2222
import math
import random
file = '/nfs/gns/literature/Santosh_Tirunagari/EBI standard Dataset/NER/list_pmc_ids.csv'
percentage=0.70
iter = 0

trainPMCids = []
devPMCids = []
testPMCids =[]
try:
    with open(file, 'r',encoding="utf-8") as fin:
        allPMCids = fin.readlines()
except:
    with open('/mnt/droplet'+file, 'r',encoding="utf-8") as fin:
        allPMCids = fin.readlines()    
    
nLines = sum(1 for line in allPMCids)
nTrain = int(nLines*percentage) 
nValid = math.floor((nLines - nTrain)/2)
nTest = nLines - (nTrain+nValid)

deck = list(range(0, nLines))
random.seed(2222) # Please dont change the seed for the reproducibility 
random.shuffle(deck)

train_ids = deck[0:nTrain]
devel_ids = deck[nTrain:nTrain+nValid]
test_ids = deck[nTrain+nValid:nTrain+nValid+nTest]

for each_pmc_id in allPMCids:
    if iter in train_ids:
        trainPMCids.append(each_pmc_id.strip())
    elif iter in devel_ids:
        devPMCids.append(each_pmc_id.strip())
    else:
        testPMCids.append(each_pmc_id.strip())

    iter = iter+1    

In [3]:
def get_Json_through_PMCID(pmcid):
    
    base_url = "https://www.ebi.ac.uk/europepmc/annotations_api/"
    article_url = urljoin(base_url, "annotationsByArticleIds?articleIds=PMC%3A"+pmcid+"&provider=Europe%20PMC&format=JSON")
    r = requests.get(article_url)
    
    if r.status_code == 200:
        return r
    else:
        return False
    

In [4]:
# result_path = '/mnt/droplet/nfs/gns/literature/Santosh_Tirunagari/EPMC Annotations Dataset/'
result_path = '/nfs/gns/literature/Santosh_Tirunagari/EPMC Annotations Dataset/'

with open(result_path+'EPMC_annotations_.csv','w',  newline='\n') as f1:
    test_writer=csv.writer(f1, delimiter='\t',lineterminator='\n')
    
    for each_test_pmc_id in testPMCids:
        ss = get_Json_through_PMCID(each_test_pmc_id[3:]) # Just the number is needed. SO remove the PMC from the front
        if ss:
            json_results = ss.json()
            pmc_id = json_results[0]['pmcid']
            print(pmc_id)
            for each_annotation in json_results[0]['annotations']:
                exact = each_annotation['prefix']+ each_annotation['exact']+each_annotation['postfix']
                token = each_annotation['tags'][0]['name']
                ner = each_annotation['type']

                row = [pmc_id, exact, token, ner]
                test_writer.writerow(row)
        else:
            continue    
            
# PMC4556948
# PMC3651197
# PMC5502978
# PMC5259676
# PMC3613406
# PMC3972685
# PMC4618948
# PMC1762380
# PMC4540425
# PMC3960246
# PMC5770482
# PMC4766309
# PMC3031208
# PMC3316545
# PMC5510223
# PMC2727484
# PMC3174205
# PMC2761781
# PMC5666160
# PMC6037156
# PMC1971115
# PMC4352028
# PMC3844564
# PMC4244103
# PMC4753424
# PMC4697806
# PMC3029330
# PMC5078810

PMC3174205
PMC4552872
PMC5972578
PMC5006041
PMC4978644
PMC5891899
PMC5120353
PMC3291930
PMC4768280
PMC4302291
PMC3949526
PMC5731848
PMC4244103
PMC3611597
PMC4556948
PMC5962829
PMC3024232
PMC3938772
PMC2474741
PMC3260253
PMC4973533
PMC6008929
PMC2478677
PMC4791522
PMC4753424
PMC4697806
PMC3029330
PMC5078810


In [5]:
manual_annot_csv = pd.read_csv('/nfs/gns/literature/Santosh_Tirunagari/EBI standard Dataset/CSV/manual_annot_exacts_180.csv', names=['pmc_id', 'sent_id', 'sentence','ner','relation'], sep ='\t')
manual_annot_csv = manual_annot_csv[manual_annot_csv['ner']!= 'No-Ner']

EPMC_annot_csv = pd.read_csv(result_path+'EPMC_annotations_.csv', names=['pmc_id', 'sentence','token','ner'], sep ='\t')

In [6]:
def sentences_tags(pmc_id, EPMC_annot_csv,manual_annot_csv):
    all_europe_pm_sentences = EPMC_annot_csv[EPMC_annot_csv['pmc_id'] == pmc_id]['sentence'].tolist()
    manual_annotated_sentences = manual_annot_csv[manual_annot_csv['pmc_id'] == pmc_id]['sentence'].tolist()
    
    full_sentences = []
    for each_sentence in tqdm(all_europe_pm_sentences):
        try:
            res = [x for x in manual_annotated_sentences if re.search(re.escape(each_sentence), x)]
            full_sentences.append(res[0])
        except:
            full_sentences.append('None')
    
    new_epmc = EPMC_annot_csv[EPMC_annot_csv['pmc_id'] == pmc_id]
    new_epmc = new_epmc.assign(full_sentence = full_sentences)
    new_epmc['combined'] = new_epmc.apply(lambda x: list([x['token'],x['ner']]),axis=1) 
    sent_tags = new_epmc.groupby('full_sentence')['combined'].apply(list).reset_index(name='tags')
    
    new_man_annot = manual_annot_csv[manual_annot_csv['pmc_id'] == pmc_id]
    new_man_annot = manual_annot_csv[manual_annot_csv['pmc_id'] == pmc_id].reset_index()
    new_man_annot.rename(columns={'sentence':'full_sentence'}, inplace=True)
    epmc_sentence_tags = pd.merge(new_man_annot, sent_tags, on='full_sentence', how='left') 

    
    return epmc_sentence_tags
        
        

In [85]:
# count = 0

path_to_csv_annotationAPI_sentences = '/mnt/droplet/nfs/gns/literature/machine-learning/evaluation/300articles/europePMC-NER/annotations_API/partial_sentences/EPMC_annotations.csv'
path_to_csv_full_sentences = '/mnt/droplet/nfs/gns/literature/machine-learning/evaluation/300articles/europePMC-NER/annotations_API/full_sentences/full_sentences_from_xmls.csv'


annotation_sent_csv = pd.read_csv(path_to_csv_annotationAPI_sentences, names=['pmc_id', 'sentence', 'token', 'ner'],
                                  sep='\t')
full_sent_csv = pd.read_csv(path_to_csv_full_sentences, names=['pmc_id', 'section', 'sentence'],
                            sep='\t')

In [86]:
each_pmc_id = 'PMC1187897'

pmc_id_annotation_sents = annotation_sent_csv[annotation_sent_csv['pmc_id'] == each_pmc_id]
pmc_id_full_sentences = full_sent_csv[full_sent_csv['pmc_id'] == each_pmc_id]
pmc_id_full_sentences_list = pmc_id_full_sentences['sentence'].tolist()

In [103]:
full_sentences_match = []

for iter_a, rows_a in pmc_id_annotation_sents.iterrows():
    try:
        res = [x for x in pmc_id_full_sentences_list if re.search(re.escape(rows_a['sentence']), x)]
        full_sentences_match.append(res[0])
    except:
        full_sentences_match.append('None')
 
pmc_id_annotation_sents = pmc_id_annotation_sents.assign(full_sentence=full_sentences_match)
pmc_id_annotation_sents

,pmc_id,sentence,token,ner,full_sentence
56956,PMC1187897,Otitis media in young Aboriginal children from...,Otitis media,Diseases,Otitis media in young Aboriginal children from...
56957,PMC1187897,Middle ear disease (otitis media) is common an...,otitis media,Diseases,Middle ear disease (otitis media) is common an...
56958,PMC1187897,revalence of middle ear disease (otitis media)...,ear disease,Diseases,The aim of the study was to determine the prev...
56959,PMC1187897,middle ear disease (otitis media) in a high-ri...,otitis media,Diseases,The aim of the study was to determine the prev...
56960,PMC1187897,Otitis media (OM) and perforation of the tympa...,Otitis media,Diseases,Otitis media (OM) and perforation of the tympa...
56961,PMC1187897,ion of the tympanic membrane (TM) were diagnosed,membrane,Gene Ontology,Otitis media (OM) and perforation of the tympa...
56962,PMC1187897,TM or recent perforation) to diagnose acute ot...,acute otitis media,Diseases,We used otoscopic criteria (bulging TM or rece...
56963,PMC1187897,Otitis media affected nearly all children (91%...,Otitis media,Diseases,Otitis media affected nearly all children (91%...
56964,PMC1187897,12) for unilateral otitis media with effusion...,otitis media with effusion,Diseases,Overall prevalence estimates adjusted for clus...
56965,PMC1187897,"(95%CI 23, 30) for acute otitis media without...",acute otitis media,Diseases,Overall prevalence estimates adjusted for clus...


In [101]:
res[1]

'In this high-risk population, high rates of tympanic perforation were associated with high rates of bulging of the tympanic membrane. '

In [ ]:
for each_pmc_id in tqdm(PMC_ids):
    # count = count + 1
    # print(each_pmc_id + '\t' + str(count))

    pmc_id_annotation_sents = annotation_sent_csv[annotation_sent_csv['pmc_id'] == each_pmc_id]
    pmc_id_full_sentences = full_sent_csv[full_sent_csv['pmc_id'] == each_pmc_id]
    pmc_id_full_sentences_list = pmc_id_full_sentences['sentence'].tolist()

    full_sentences_match = []

    for iter_a, rows_a in pmc_id_annotation_sents.iterrows():
        try:
            res = [x for x in pmc_id_full_sentences_list if re.search(re.escape(rows_a['sentence']), x)]
            if not res[0] in full_sentences_match:
                full_sentences_match.append(res[0])
            else:
                full_sentences_match.append(res[1])
        except:
            full_sentences_match.append('None')

    if full_sentences_match:
        pmc_id_annotation_sents = pmc_id_annotation_sents.assign(full_sentence=full_sentences_match)
        pmc_id_annotation_sents['combined'] = pmc_id_annotation_sents.apply(lambda x: list([x['sentence'], x['token'], x['ner']]),
                                                                                    axis=1)
        sent_tags = pmc_id_annotation_sents.groupby('full_sentence')['combined'].apply(list).reset_index(name='tags')
        pmc_id_full_sentences.rename(columns={'sentence': 'full_sentence'}, inplace=True)
        epmc_full_sentence_tags = pd.merge(pmc_id_full_sentences, sent_tags, on='full_sentence', how='left')

In [7]:
from ast import literal_eval

def find_sub_list(sl,l):
    results=[]
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            results.append((ind,ind+sll))

    return results


def convert2IOB_dict_manual_annotations(text_data,ner_tags):
    
    tokens = []
    ners = []
    
    split_text = wordpunct_tokenize(text_data)
    # for each word token append 'O'
    arr = ['O']*len(split_text) 
    
    if ner_tags != 'No-Ner':
        ner_tags = literal_eval(ner_tags)
        for each_tag in ner_tags:
            token_list = wordpunct_tokenize(each_tag[2])
            ner_list = wordpunct_tokenize(each_tag[3])

            if(len(token_list) > len(ner_list)):
                ner_list = len(token_list) * ner_list

            for i in range(0,len(ner_list)):
                # The logic here is look for the first B-tag and then append I-tag next
                if(i==0):
                    ner_list[i] = 'B-'+ner_list[i]
                else:
                    ner_list[i] = 'I-'+ner_list[i]

            tokens.append(token_list)
            ners.append(ner_list)

        for i in range(0, len(tokens)):
            spans = find_sub_list(tokens[i], split_text)
            for each_span in spans:
                arr[each_span[0]:each_span[1]] = ners[i]
    
        return zip(split_text, arr)

    else:
        return zip(split_text, arr)
            
            
    
    
def convert2IOB_dict_EPMC_annotations(text_data,ner_tags):
    
    tokens = []
    ners = []
    
    split_text = wordpunct_tokenize(text_data)
    # for each word token append 'O'
    arr = ['O']*len(split_text)
        

    if ner_tags != 'No-Ner':
        for each_tag in ner_tags:
            token_list = wordpunct_tokenize(each_tag[0])
            ner_list = wordpunct_tokenize(each_tag[1])

            if(len(token_list) > len(ner_list)):
                ner_list = len(token_list) * ner_list

            for i in range(0,len(ner_list)):
                # The logic here is look for the first B-tag and then append I-tag next
                if(i==0):
                    ner_list[i] = 'B-'+ner_list[i]
                else:
                    ner_list[i] = 'I-'+ner_list[i]

            tokens.append(token_list)
            ners.append(ner_list)

        for i in range(0, len(tokens)):
            spans = find_sub_list(tokens[i], split_text)
            for each_span in spans:
                arr[each_span[0]:each_span[1]] = ners[i]

        return zip(split_text, arr)
    else:
        return zip(split_text, arr)

In [8]:
from nltk.tokenize import wordpunct_tokenize

result_path = '/nfs/gns/literature/Santosh_Tirunagari/EPMC Annotations Dataset/'

with open(result_path+'EPMC_annotated_test.csv','w',  newline='\n') as f1, open(result_path+'manual_annotated_test.csv','w',  newline='\n') as f2: 
    epmc_writer=csv.writer(f1, delimiter='\t',lineterminator='\n')
    manual_writer=csv.writer(f2, delimiter='\t',lineterminator='\n')
    
    for each_pmc_id in testPMCids:
        print(each_pmc_id)
        ss_ = sentences_tags(each_pmc_id, EPMC_annot_csv,manual_annot_csv)
        ss = ss_.where((pd.notnull(ss_)), 'No-Ner')
        for index, row in tqdm(ss.iterrows(), total=ss.shape[0]):
            tagged_tokens = convert2IOB_dict_EPMC_annotations(row['full_sentence'],row['tags'])
            for each_word in tagged_tokens: # make it to manual annotation format for easy comparison
                epmc_tokens = list(each_word)
                if epmc_tokens[1] == 'B-Diseases':
                    epmc_tokens[1] = 'B-DS'
                elif epmc_tokens[1] == 'I-Diseases':
                    epmc_tokens[1] = 'I-DS'
                elif epmc_tokens[1] == 'B-Organisms':
                    epmc_tokens[1] = 'B-OG'
                elif epmc_tokens[1] == 'I-Organisms':
                    epmc_tokens[1] = 'I-OG'  
                elif epmc_tokens[1] == 'B-Gene_Proteins':
                    epmc_tokens[1] = 'B-GP'   
                elif epmc_tokens[1] == 'I-Gene_Proteins':
                    epmc_tokens[1] = 'I-GP'  
                else:
                    epmc_tokens[1] = 'O'
                epmc_writer.writerow(epmc_tokens)
            epmc_writer.writerow('') 
            manual_tagged_tokens = convert2IOB_dict_manual_annotations(row['full_sentence'],row['ner'])
            for each_word in manual_tagged_tokens:
                manual_writer.writerow(list(each_word))
            manual_writer.writerow('') 

 28%|██▊       | 109/390 [00:00<00:00, 1081.65it/s]

PMC3174205


  0%|          | 0/215 [00:00<?, ?it/s]

PMC4552872
PMC5972578


100%|██████████| 217/217 [00:00<00:00, 1464.67it/s]


...

PMC4753424
PMC4697806


 13%|█▎        | 46/362 [00:00<00:00, 450.38it/s]

PMC3029330


100%|██████████| 160/160 [00:00<00:00, 481067.13it/s]
0it [00:00, ?it/s]

PMC5078810


In [9]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

true_lbs = pd.read_csv(result_path+'manual_annotated_test.csv', sep='\t', names=['tokens','tags'])
pred_lbs = pd.read_csv(result_path+'EPMC_annotated_test.csv', sep='\t', names=['tokens','tags'])

y_true = true_lbs['tags'].values
y_pred =  pred_lbs['tags'].values


class_labels = sorted([tag for tag in set(y_true) if tag != 'O'], key=lambda name: (name[1:], name[0]))

from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, labels = class_labels))


#               precision    recall  f1-score   support

#         B-DS       0.74      0.53      0.62      1323
#         I-DS       0.84      0.70      0.76       486
#         B-GP       0.76      0.52      0.62      3056
#         I-GP       0.89      0.41      0.56      1989
#         B-OG       0.93      0.69      0.79      1960
#         I-OG       0.98      0.51      0.68       968

#    micro avg       0.83      0.54      0.66      9782
#    macro avg       0.85      0.56      0.67      9782
# weighted avg       0.84      0.54      0.65      9782

              precision    recall  f1-score   support

        B-DS       0.64      0.56      0.60      1154
        I-DS       0.70      0.60      0.65       475
        B-GP       0.85      0.64      0.73      3246
        I-GP       0.91      0.37      0.52      2243
        B-OG       0.91      0.76      0.83      2374
        I-OG       0.95      0.77      0.85      1185

   micro avg       0.85      0.61      0.71     10677
   macro avg       0.82      0.62      0.69     10677
weighted avg       0.85      0.61      0.70     10677



In [9]:
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

result_path = '/nfs/gns/literature/Santosh_Tirunagari/EPMC Annotations Dataset/'

true_lbs = pd.read_csv(result_path+'manual_annotated_test.csv', sep='\t', names=['tokens','tags'])
pred_lbs = pd.read_csv(result_path+'EPMC_annotated_test.csv', sep='\t', names=['tokens','tags'])

true_lbs['tags'].replace('B-|I-','',regex=True, inplace=True)
pred_lbs['tags'].replace('B-|I-','',regex=True, inplace=True)

y_true = true_lbs['tags'].values
y_pred =  pred_lbs['tags'].values


class_labels = sorted([tag for tag in set(y_true) if tag != 'O'], key=lambda name: (name[1:], name[0]))
print(classification_report(y_true, y_pred, labels = class_labels))

              precision    recall  f1-score   support

          OG       0.93      0.77      0.84      3559
          GP       0.87      0.54      0.66      5489
          DS       0.70      0.62      0.66      1629

   micro avg       0.86      0.62      0.72     10677
   macro avg       0.84      0.64      0.72     10677
weighted avg       0.87      0.62      0.72     10677



In [4]:
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

bern_path = '/nfs/gns/literature/machine-learning/Shyama/Bern_NER/IOB_result/'
manual_path = '/nfs/gns/literature/machine-learning/Shyama/manual_2batches/IOB_2batches/'

true_lbs = pd.read_csv(manual_path+'data_IOB.csv', sep='\t', names=['tokens','tags'])
pred_lbs = pd.read_csv(bern_path+'bern_IOB.csv', sep='\t', names=['tokens','tags'])

true_lbs['tags'].replace('B-|I-','',regex=True, inplace=True)
pred_lbs['tags'].replace('B-|I-','',regex=True, inplace=True)

y_true = true_lbs['tags'].values
y_pred =  pred_lbs['tags'].values


class_labels = sorted([tag for tag in set(y_true) if tag != 'O'], key=lambda name: (name[1:], name[0]))
print(classification_report(y_true, y_pred, labels = class_labels))

KeyboardInterrupt: 

In [82]:
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report


bern_path = '/mnt/droplet/nfs/gns/literature/machine-learning/Shyama/Bern_NER/IOB_result/'
manual_path = '/mnt/droplet/nfs/gns/literature/machine-learning/Datasets/NER_Datasets/EBI_standard-IOB/'


true_lbs = pd.read_csv(manual_path+'test.csv', sep='\t', names=['tokens','tags'])
pred_lbs = pd.read_csv(bern_path+'test.csv', sep='\t', names=['tokens','tags'])

y_true = true_lbs['tags'].values
y_pred =  pred_lbs['tags'].values


class_labels = sorted([tag for tag in set(y_true) if tag != 'O'], key=lambda name: (name[1:], name[0]))

from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, labels = class_labels))

ValueError: Found input variables with inconsistent numbers of samples: [358242, 138092]

In [16]:
import glob
import os

print('Evaluating EPMC vs ML NER !! Please be patient')
result_path = '/mnt/droplet/nfs/gns/literature/machine-learning/EPMC_annotations/'
xml_path = '/mnt/droplet/nfs/gns/literature/machine-learning/benchmarking/benchmark_sent/'
all_files = sorted(glob.glob(xml_path + '*.xml'))

names = [os.path.basename(x) for x in all_files]
PMC_ids = [x[:-4] for x in names]

len(names)

Evaluating EPMC vs ML NER !! Please be patient


2000

In [13]:
import os
import pandas as pd
import glob
import json
import csv
import sys

import multiprocessing

import numpy as np
import re

from nltk.tokenize import wordpunct_tokenize

import requests
# from pprint import pprint
import pandas as pd

from collections import defaultdict, Counter
import time
from requests.compat import urljoin

from tqdm import tqdm
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

In [14]:
def get_Json_through_PMCID(pmcid):
    base_url = "https://www.ebi.ac.uk/europepmc/annotations_api/"
    article_url = urljoin(base_url,
                          "annotationsByArticleIds?articleIds=PMC%3A" + pmcid + "&provider=Europe%20PMC&format=JSON")
    r = requests.get(article_url)

    if r.status_code == 200:
        return r
    else:
        return False


def get_epmc_annotations_to_file(result_path, PMCids):
    # result_path = '/nfs/gns/literature/Santosh_Tirunagari/EPMC Annotations Dataset/'

    with open(result_path + 'EPMC_annotations.csv', 'w', newline='\n') as f1:
        test_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

        count = 0
        for each_test_pmc_id in PMCids:
            count = count+1
            print(each_test_pmc_id + '\t' + str(count))
            ss = get_Json_through_PMCID(each_test_pmc_id[3:])  # Just the number is needed. So remove the PMC from the front
            if ss:
                json_results = ss.json()
                pmc_id = json_results[0]['pmcid']
                print(pmc_id)
                for each_annotation in json_results[0]['annotations']:
                    exact = each_annotation['prefix'] + each_annotation['exact'] + each_annotation['postfix']
                    token = each_annotation['tags'][0]['name']
                    ner = each_annotation['type']

                    row = [pmc_id, exact, token, ner]
                    test_writer.writerow(row)
            else:
                continue

In [4]:
import pandas as pd
path_to_csv_annotationAPI_sentences = '/mnt/droplet/nfs/gns/literature/machine-learning/evaluation/300articles/ML-NER/annotations_API_sentenes/partial_sentences/EPMC_annotations.csv'
path_to_csv_full_sentences = '/mnt/droplet/nfs/gns/literature/machine-learning/evaluation/300articles/ML-NER/annotations_API_sentenes/full_sentences/europepmc/full_sentences_from_xmls.csv'
xml_path = '/mnt/droplet/nfs/gns/literature/machine-learning/benchmarking/benchmark_sent/'



annotation_sent_csv = pd.read_csv(path_to_csv_annotationAPI_sentences, names=['pmc_id', 'sentence', 'token', 'ner'], sep='\t')
full_sent_csv =  pd.read_csv(path_to_csv_full_sentences, names=['pmc_id', 'type', 'sentence'],
                             sep='\t')


all_files = sorted(glob.glob(xml_path + '*.xml'))
names = [os.path.basename(x) for x in all_files]
PMC_ids = [x[:-4] for x in names]



In [13]:
each_pmc_id = 'PMC4556948'
pmc_id_annotation_sents = annotation_sent_csv[annotation_sent_csv['pmc_id'] == each_pmc_id]
pmc_id_full_sentences = full_sent_csv[full_sent_csv['pmc_id'] == each_pmc_id]

pmc_id_full_sentences_list = pmc_id_full_sentences['sentence'].tolist()

In [25]:
full_sentences_match =[]
for iter_a, rows_a in pmc_id_annotation_sents.iterrows():
    try:
        res = [x for x in pmc_id_full_sentences_list if re.search(re.escape(rows_a['sentence']), x)]
        full_sentences_match.append(res[0])
    except:
        full_sentences_match.append('None')

full_sentences_match        

['GR3027 antagonizes GABAA receptor-potentiating neurosteroids and restores spatial learning and motor coordination in rats with chronic hyperammonemia and hepatic encephalopathy ',
 'GR3027 antagonizes GABAA receptor-potentiating neurosteroids and restores spatial learning and motor coordination in rats with chronic hyperammonemia and hepatic encephalopathy ',
 'GR3027 antagonizes GABAA receptor-potentiating neurosteroids and restores spatial learning and motor coordination in rats with chronic hyperammonemia and hepatic encephalopathy ',
 'Hepatic encephalopathy (HE) is one of the primary complications of liver cirrhosis. ',
 'Hepatic encephalopathy (HE) is one of the primary complications of liver cirrhosis. ',
 'Current treatments for HE, mainly directed to reduction of ammonia levels, are not effective enough because they cannot completely eliminate hyperammonemia and inflammation, which induce the neurological alterations. ',
 'Current treatments for HE, mainly directed to reduct

In [26]:
new_pmc_id_annotation_sents = pmc_id_annotation_sents
new_pmc_id_annotation_sents = new_pmc_id_annotation_sents.assign(full_sentence=full_sentences_match)
new_pmc_id_annotation_sents

,pmc_id,sentence,token,ner,full_sentence
0,PMC4556948,nd restores spatial learning and motor coordinat,learning,Gene Ontology,GR3027 antagonizes GABAA receptor-potentiating...
1,PMC4556948,tor coordination in rats with chronic hypera,rats,Organisms,GR3027 antagonizes GABAA receptor-potentiating...
2,PMC4556948,n rats with chronic hyperammonemia and hepatic...,hyperammonemia,Diseases,GR3027 antagonizes GABAA receptor-potentiating...
3,PMC4556948,Hepatic encephalopathy (HE) is one of the prim...,Hepatic encephalopathy,Diseases,Hepatic encephalopathy (HE) is one of the prim...
4,PMC4556948,s one of the primary complications of liver ci...,liver cirrhosis,Diseases,Hepatic encephalopathy (HE) is one of the prim...
5,PMC4556948,"ted to reduction of ammonia levels, are not eff",ammonia,Chemicals,"Current treatments for HE, mainly directed to ..."
6,PMC4556948,ompletely eliminate hyperammonemia and inflamm...,hyperammonemia,Diseases,"Current treatments for HE, mainly directed to ..."
7,PMC4556948,"ordination, spatial learning, and circadian rhyt",learning,Gene Ontology,This work aimed to assess whether GR3027 impro...
8,PMC4556948,"atial learning, and circadian rhythms of activ...",circadian rhythms,Gene Ontology,This work aimed to assess whether GR3027 impro...
9,PMC4556948,cadian rhythms of activity in rats with HE.,rats,Organisms,This work aimed to assess whether GR3027 impro...


In [114]:
result_path = '/mnt/droplet/nfs/gns/literature/machine-learning/EPMC_annotations/full_sentences/'
pmc_id_full_sentences.to_csv(result_path+'Europe_PMC_annotation.csv', mode='a', header=False, sep='\t', index=False)

In [27]:
if full_sentences_match:
    new_pmc_id_annotation_sents = pmc_id_annotation_sents
    new_pmc_id_annotation_sents = new_pmc_id_annotation_sents.assign(full_sentence=full_sentences_match)
    new_pmc_id_annotation_sents['combined'] = new_pmc_id_annotation_sents.apply(lambda x: list([x['token'], x['ner']]), axis=1)
    sent_tags = new_pmc_id_annotation_sents.groupby('full_sentence')['combined'].apply(list).reset_index(name='tags')
    pmc_id_full_sentences.rename(columns={'sentence': 'full_sentence'}, inplace=True)
    epmc_full_sentence_tags = pd.merge(new_man_annot, sent_tags, on='full_sentence', how='left')


/home/stirunag/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


NameError: name 'new_man_annot' is not defined

In [30]:
for each_pmc_id in tqdm(PMC_ids):
    # count = count + 1
    # print(each_pmc_id + '\t' + str(count))

    pmc_id_annotation_sents = annotation_sent_csv[annotation_sent_csv['pmc_id'] == each_pmc_id]
    pmc_id_full_sentences = full_sent_csv[full_sent_csv['pmc_id'] == each_pmc_id]
    pmc_id_full_sentences_list = pmc_id_full_sentences['sentence'].tolist()

    full_sentences_match = []

    for iter_a, rows_a in pmc_id_annotation_sents.iterrows():
        try:
            res = [x for x in pmc_id_full_sentences_list if re.search(re.escape(rows_a['sentence']), x)]
            if not res[0] in full_sentences_match:
                full_sentences_match.append(res[0])
            else:
                full_sentences_match.append(res[1])
        except:
            full_sentences_match.append('None')

    if full_sentences_match:
        pmc_id_annotation_sents = pmc_id_annotation_sents.assign(full_sentence=full_sentences_match)
        pmc_id_annotation_sents['combined'] = pmc_id_annotation_sents.apply(lambda x: list([x['sentence'], x['token'], x['ner']]),
                                                                                    axis=1)
        sent_tags = pmc_id_annotation_sents.groupby('full_sentence')['combined'].apply(list).reset_index(name='tags')
        pmc_id_full_sentences.rename(columns={'sentence': 'full_sentence'}, inplace=True)
        epmc_full_sentence_tags = pd.merge(pmc_id_full_sentences, sent_tags, on='full_sentence', how='left')



100%|██████████| 2000/2000 [00:18<00:00, 105.89it/s]


In [66]:
text_data = epmc_full_sentence_tags['full_sentence'].iloc[134]
ner_tags = epmc_full_sentence_tags['tags'].iloc[134]

In [80]:
epmc_full_sentence_tags['tags'].iloc[23]
epmc_full_sentence_tags['full_sentence'].iloc[23]

'Thus, FGF21 may be an intermediary between the clock components, metabolism and reproductive fitness. '

In [68]:
span_text = list(tokenizer.span_tokenize(text_data))
split_text = tokenizer.tokenize(text_data)
split_token_span_list = list(zip(split_text, span_text))
split_token_span_list

[('In', (0, 2)),
 ('the', (3, 6)),
 ('liver', (7, 12)),
 (',', (12, 13)),
 ('the', (14, 17)),
 ('tissue', (18, 24)),
 ('-', (24, 25)),
 ('specific', (25, 33)),
 ('factor', (34, 40)),
 ('is', (41, 43)),
 ('HNF6', (44, 48)),
 ('through', (49, 56)),
 ('which', (57, 62)),
 ('REV', (63, 66)),
 ('-', (66, 67)),
 ('ERBα', (67, 71)),
 ('can', (72, 75)),
 ('modulate', (76, 84)),
 ('gene', (85, 89)),
 ('expression', (90, 100)),
 ('in', (101, 103)),
 ('an', (104, 106)),
 ('HNF6', (107, 111)),
 ('-', (111, 112)),
 ('dependent', (112, 121)),
 ('fashion', (122, 129)),
 ('.', (129, 130))]

In [72]:
ner_tags

[['-specific factor is HNF6 through which REV-E', 'HNF6', 'Gene_Proteins']]

In [70]:
each_tag = ner_tags[0]
sub_text_list = wordpunct_tokenize(each_tag[0])
sub_text_list

['-', 'specific', 'factor', 'is', 'HNF6', 'through', 'which', 'REV', '-', 'E']

In [71]:
token = each_tag[1]
token

'HNF6'

In [ ]:
for each_

In [54]:
each_tag = ner_tags[0]

sub_text_list = wordpunct_tokenize(each_tag[0])
token_list = wordpunct_tokenize(each_tag[1])
ner_list = [each_tag[2]]

In [55]:
if (len(token_list) > len(ner_list)):
    ner_list = len(token_list) * ner_list
for i in range(0, len(ner_list)):
    # The logic here is look for the first B-tag and then append I-tag next
    if (i == 0):
        ner_list[i] = 'B-' + ner_list[i]
    else:
        ner_list[i] = 'I-' + ner_list[i]

['B-Gene', 'I-Ontology']

In [34]:
from nltk.tokenize import WordPunctTokenizer

def convert2IOB_EPMC_annotations(text_data, ner_tags):

    tokenizer = WordPunctTokenizer()

    tokens = []
    ners = []
    spans = []

    split_text = tokenizer.tokenize(text_data)
    span_text = list(tokenizer.span_tokenize(text_data))
    # for each word token append 'O'
    arr = ['O'] * len(split_text)
    

    if ner_tags:
        try:
            ner_tags = literal_eval(ner_tags)
        except:
            pass

    elif isinstance(ner_tags, float)  and ner_tags is None:
        return zip(split_text, arr)
    else:
        return zip(split_text, arr)

    for each_tag in ner_tags:
        if each_tag[2] in ("Gene_Proteins", "Diseases", "Organisms"):

            if each_tag[2] == 'Gene_Proteins':
                each_tag[2] = 'GP'
            elif each_tag[2] == 'Diseases':
                each_tag[2] = 'DS'
            if each_tag[2] == 'Organisms':
                each_tag[2] = 'OG'

            sub_text_list = wordpunct_tokenize(each_tag[0])
            token_list = wordpunct_tokenize(each_tag[1])
            ner_list = wordpunct_tokenize(each_tag[2])

            if (len(token_list) > len(ner_list)):
                ner_list = len(token_list) * ner_list
            for i in range(0, len(ner_list)):
                # The logic here is look for the first B-tag and then append I-tag next
                if (i == 0):
                    ner_list[i] = 'B-' + ner_list[i]
                else:
                    ner_list[i] = 'I-' + ner_list[i]
            tokens.append(token_list)
            ners.append(ner_list)

        for i in range(0, len(tokens)):
            spans = find_sub_list(tokens[i], split_text)
            for each_span in spans:
                arr[each_span[0]:each_span[1]] = ners[i]

    return zip(split_text, arr)

[[' functions including metabolism and reproduction. ',
  'metabolism',
  'Gene Ontology']]